In [32]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression

%matplotlib inline

In [2]:
PROCESSED_DATA_FILE = './../data/project_brazil/processed.csv'

RANDOM_STATE = 14

In [3]:
df = pd.read_csv(PROCESSED_DATA_FILE)

In [4]:
def prcp_transform(val):
    if val == 0.0:
        return 0
    elif (val > 0.0) & (val <= 1.2):
        return 1
    elif (val > 1.2) & (val <=2.5):
        return 2
    elif (val > 2.5):
        return 3

---

Подготовим датасет для дальнейшей работы моделей. Начнем с целевой переменной. Выполним фильтрацию всего датасета по 99% квантилю целевой перемнной. Выполним сдвиг ее значений на одну позицию вперед, чтобы прогнозировать осадки на час, следующий за наблюдаемым, и выкинем из датасета первую строчку, у которой значение признака **prcp** станет пустым.

In [5]:
prcp_quantile = df['prcp'].quantile(0.99)
df = df[df['prcp'] < prcp_quantile]

df['prcp'] = df['prcp'].shift(1)
df.drop(index=0, inplace=True)

Выполним кодирование значений целевой переменной и выделим ее в отдельный объект.

In [6]:
y = df['prcp'].apply(prcp_transform).astype('int')

Создадим словарь обработанных признаков, для дальнейшего их объединения в итоговый датасет.

In [44]:
processed_features = dict()

<p>Т.к. одной из моделей у нас является линейная, то категориальные признаки нам необходимо закодировать, а численные - отмасштабировать.
<p>Обозначим схему преобразования данных. В нашей задаче у нас отсутствует тестовая выборка, валидацию моделей будем проводить на отложенной части выборки. Но преобразовывать признаки мы будем на всей имеющейся выборке сразу. В общем случае это неправильно, т.к. из валидационной части может "просочиться" информация в обучающую часть (например, учтется масштаб всех значений признаков, будут закодированы все значения категориальных признаков, и т.д.). Мы же закроем глаза на это допущение.

Признак **wsid**. Является категориальным, выполним One-Hot-Encoding с выбрасыванием первой колонки (для того, чтобы не возникало зависимости в новых категориальных признаках). Признаки **wsnm, inme** не используем как дублирующие.

In [45]:
wsid_ohe = pd.get_dummies(df['wsid'], prefix='wsid_ohe', drop_first=True)
processed_features['wsid_ohe'] = wsid_ohe

Признаки **elvt, lat, lon** являются численными, выполним масштабирование.

In [46]:
stnd_scaler = StandardScaler()

elvt_scaled = stnd_scaler.fit_transform(df[['elvt']])
processed_features['elvt_scaled'] = pd.DataFrame(elvt_scaled, columns=['elvt_scaled'])

lat_scaled = stnd_scaler.fit_transform(df[['lat']])
processed_features['lat_scaled'] = pd.DataFrame(lat_scaled, columns=['lat_scaled'])

lon_scaled = stnd_scaler.fit_transform(df[['lon']])
processed_features['lon_scaled'] = pd.DataFrame(lon_scaled, columns=['lon_scaled'])

Признаки **city, prov** не будем использовать в моделях, как малоинформативные (географическую информацию мы уже добавили в виде широты и долготы), но потенциально "раздувающие" датасет (они категориальны и придется кодировать).

Как было обозначено ранее, из признаков даты и времени мы будем использовать только **mo** (месяц). Пояснения см. в п. 4. Несмотря на целочисленность признака, он является конечно же категориальным, так что выполним One-Hot-Encoding.

In [47]:
mo_ohe = pd.get_dummies(df['mo'], prefix='mo_ohe', drop_first=True)
processed_features['mo_ohe'] = mo_ohe

Для **stp, temp, dewp** (давления, температуры воздуха, температуры точки росы) возьмем только мгновенные их значения (максимальные и минимальные значения линейно зависимы, см. п. 4). Воспользуемся масштабированием. Пустые значения заменим средними.

In [48]:
stp_scaled = stnd_scaler.fit_transform(df[['stp']])
processed_features['stp_scaled'] = pd.DataFrame(stp_scaled, columns=['stp_scaled'])

temp_scaled = stnd_scaler.fit_transform(df[['temp']].fillna(df[['temp']].mean()))
processed_features['temp_scaled'] = pd.DataFrame(temp_scaled, columns=['temp_scaled'])

dewp_scaled = stnd_scaler.fit_transform(df[['dewp']].fillna(df[['dewp']].mean()))
processed_features['dewp_scaled'] = pd.DataFrame(dewp_scaled, columns=['dewp_scaled'])

Для **gbrd** (солнечное излучение) заполним пропуски и выполним масштабирование.

In [49]:
gbrd_scaled = stnd_scaler.fit_transform(df[['gbrd']].fillna(df[['gbrd']].mean()))
processed_features['gbrd_scaled'] = pd.DataFrame(gbrd_scaled, columns=['gbrd_scaled'])

Показатели относительной влажности (мгновенная, максимальная и минимальная) используем все, т.к. линейной зависимости в них не наблюдалось, и данные могут быть полезными для прогнозирования осадков. Воспользуемся масштабированием. Пустые значения заменим средними.

In [50]:
hmdy_scaled = stnd_scaler.fit_transform(df[['hmdy']])
processed_features['hmdy_scaled'] = pd.DataFrame(hmdy_scaled, columns=['hmdy_scaled'])

hmin_scaled = stnd_scaler.fit_transform(df[['hmin']].fillna(df[['hmin']].mean()))
processed_features['hmin_scaled'] = pd.DataFrame(hmin_scaled, columns=['hmin_scaled'])

hmax_scaled = stnd_scaler.fit_transform(df[['hmax']].fillna(df[['hmax']].mean()))
processed_features['hmax_scaled'] = pd.DataFrame(hmax_scaled, columns=['hmax_scaled'])

Показатели ветра **wdsp, gust** (скорость, порывы) отмасштабируем и заполним пропуски. Направление ветра **wdct** разобьем на 10-градусные сектора, 36-ой сектор заменим нулевым, и выполним One-Hot-Encoding.

In [51]:
wdsp_scaled = stnd_scaler.fit_transform(df[['wdsp']].fillna(df[['wdsp']].mean()))
processed_features['wdsp_scaled'] = pd.DataFrame(wdsp_scaled, columns=['wdsp_scaled'])

gust_scaled = stnd_scaler.fit_transform(df[['gust']].fillna(df[['gust']].mean()))
processed_features['gust_scaled'] = pd.DataFrame(gust_scaled, columns=['gust_scaled'])

wdct_processed = df['wdct'].fillna(df[['wdct']].mean())
wdct_processed = wdct_processed.apply(lambda x: x // 10).apply(lambda x: x if x != 36.0 else 0.0)
wdct_ohe = pd.get_dummies(wdct_processed, prefix='wdct_ohe', drop_first=True)
processed_features['wdct_ohe'] = wdct_ohe

Наконец, соберем все признаки воедино.

In [53]:
for name_df, part_df in processed_features.items():
    processed_features[name_df] = part_df.reset_index(drop=True)

X = pd.concat(processed_features.values(), axis=1)
print('Размер обработанной выборки: {}'.format(X.shape))

Размер обработанной выборки: (1393614, 179)


Проведем отбор признаков. Для начала обучим логистическую регрессию на всей выборке с L1-регуляризатором. Этот тип регуляризации, также называемый Lasso, характеризуется тем, что обнуляет веса у наименее значимых признаков. И чем больше параметр регуляризации, тем больше будут обнуляться веса.

In [ ]:
lr_l1_model = LogisticRegression(penalty='l1', C=1.0, random_state=RANDOM_STATE)
lr_l1_model.fit(X, y)

In [ ]:
zip(X.columns, lr_l1_model.coef_)

Выделим обучающую и валидационную части выборки. Ранее уже упоминалось, что упорядочивание выборки по времени не имеет смысла, так что воспользуемся случайным разбиением и выделим для валидации 30% всей выборки. Укажем параметр stratify для сохранения баланса классов в новых выборках.

In [16]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, shuffle=True, stratify=y, random_state=RANDOM_STATE)

In [17]:
print('Размер обучающей выборки:', X_train.shape)
print('Размер обучающего вектора целевой переменной:', y_train.shape)
print('Размер валидационной выборки:', X_test.shape)
print('Размер валидационного вектора целевой переменной:', y_test.shape)

Размер обучающей выборки: (975529, 194)
Размер обучающего вектора целевой переменной: (975529,)
Размер валидационной выборки: (418085, 194)
Размер валидационного вектора целевой переменной: (418085,)
